In [16]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from scipy.optimize import curve_fit
# from tqdm.notebook import tqd
import statsmodels.api as sm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [20, 8]

from datetime import datetime, timedelta
import time
from patsy import dmatrices
import random as rand
warnings.filterwarnings('ignore')

import pylab as pl
from IPython import display


In [17]:
POP = 10000     # assume population 1 mil
CONTACT = 4              # people in contact per day
INFECT_CHANCE = 0.054    # chance of being infected when close to the infected
FLIGHTS = 34.2/264/365   # flights per day
FLIGHTS_REDUCED = 15     # date until flights is all cancelled
ASYM = 0.505             # % of being asymptomatic while infected

HOSPITALIZATION = 0.5    # assuming Indo's hospitalisation rate is 70% of the studies mean
ICU = 0.3                # ventilator much much lower
INFECTITIUS_PERIOD = 3   # mean, exponentially assume
TIME_RECOVER = 11        # mean until recover, poisson
TIME_DEATH = 32          # mean until recover, poisson
TIME_HOSP = 7            # mean until being hospitalised, poisson

In [18]:
 age = ['0-19', '20-44', '45-54', '55-64', '65-74', '75-84', '85'] 

hosl = [ x*HOSPITALIZATION/100 for x in [1.6, 14.3, 21.2, 20.5, 28.6, 30.5, 31.3]]
hosu = [ x*HOSPITALIZATION/100 for x in [2.5, 20.8, 28.3, 30.1, 43.5, 58.7, 70.3]]

icul = [ x*ICU/100 for x in [0, 2, 5.4, 4.7, 8.1, 10.5, 6.3]]
icuu = [ x*ICU/100 for x in [0, 4.2, 10.4, 11.2, 18.8, 31, 29]]

fatl = [x/100 for x in [0, 0.1, 0.5, 1.4, 2.7, 4.3, 10.4]]
fatu = [x/100 for x in [0, 0.2, 0.8, 2.6, 4.9, 10.5, 27.3]]

indo = [433, 521, 169, 120, 67, 19, 15]

ageb = pd.DataFrame(list(zip(age,hosl,hosu,icul,icuu,fatl,fatu,indo)), 
        columns =['age','hosl','hosu','icul','icuu','fatl','fatu','indo'])


In [19]:
df = pd.DataFrame()
for i,row in ageb.iterrows():
    temp = pd.DataFrame()
    temp['age'] = [row['age']]*(( row['indo']*POP)//1329)
    df = df.append(temp)
    
df = df['age'].sample(n=POP)
data = pd.merge(left=df, right=ageb, how='left', left_on='age', right_on='age')
data = data.dropna()
data['status'] = 'S'
data.drop('indo', axis=1, inplace=True)
data.head()

,age,hosl,hosu,icul,icuu,fatl,fatu,status
0,20-44,0.0715,0.1040,0.006,0.0126,0.001,0.002,S
1,20-44,0.0715,0.1040,0.006,0.0126,0.001,0.002,S
2,0-19,0.0080,0.0125,0.000,0.0000,0.000,0.000,S
3,20-44,0.0715,0.1040,0.006,0.0126,0.001,0.002,S
4,20-44,0.0715,0.1040,0.006,0.0126,0.001,0.002,S


In [20]:
df = data.copy()
df['t'] = None;

# pre-assign timetill recover, death/recover, timetillhospital, hospitalization, ICU, 

df['asym'] = np.random.binomial(1, ASYM, len(df))
df['asym'] = df['asym'].replace(0,False).replace(1,True)
# df['timeH'] = TIME_HOSP # np.random.poisson(TIME_HOSP, len(df))
df['hosp'] = [True if not(x) & (np.random.binomial(1, y) < 0.5) else False for x,y in zip(df['asym'],df['hosl'])]
# df['timeR'] = TIME_RECOVER # np.random.poisson(TIME_RECOVER, len(df))
df['death'] = ['D' if rand.random() < np.random.binomial(1, x) else 'R' for x in df['fatu']]
# df['timeD'] = TIME_DEATH #np.random.poisson(TIME_DEATH, len(df))

In [ ]:
# time.sleep(15)
result = pd.DataFrame(columns=['age','t','Susceptible','Infected','Asymptomatic','Hospitalized','Recovered','Death'])

fig=plt.figure(figsize=(12, 7), dpi= 80, facecolor='w', edgecolor='k')

for t in range(100):
    S = df.loc[df['status']=='S'].shape[0]
    I = df.loc[df['status']=='I'].shape[0]
    A = df.loc[(df['status']=='I') & df['asym']].shape[0]
    H = df.loc[(df['status'] != 'R') & df['hosp'] & (TIME_HOSP <= t - df['t'])].shape[0]
    R = df.loc[df['status']=='R'].shape[0]
    D = df.loc[df['status']=='D'].shape[0]
    
    pl.scatter(t, I,c='#F28E2B')
    pl.scatter(t, S+I, c='#4E79A7')
    pl.scatter(t, R, c='#76B7B2')
    pl.scatter(t, A, c='#BAB0AC')
    pl.scatter(t, H, c='#9C755F')
    pl.scatter(t, D, c='#E15759')
    display.clear_output(wait=True)
    display.display(pl.gcf())
    
    for a in age:
        d = df.loc[df['age']==a]
        result.loc[len(result)] = [a, t,
                  d.loc[d['status']=='S'].shape[0],
                  d.loc[d['status']=='I'].shape[0],
                  d.loc[(d['status']=='I') & d['asym']].shape[0],
                  d.loc[(d['status'] != 'R') & d['hosp'] & (TIME_HOSP <= t - d['t'])].shape[0],
                  d.loc[d['status']=='R'].shape[0],
                  d.loc[d['status']=='D'].shape[0]
                  ]
#     print(result.loc[len(result)-1].to_list
    print(t,S,I,A,H,R,D)
    
    # cross infection
    ind =  df.index[(df['status']=='S') ].tolist()
    if len(ind):
        df.loc[ind, ['status','t']] = [['I',t] if rand.random() < INFECT_CHANCE*I*CONTACT/POP else ['S', None] for x in ind]
    
    # imported cases
    if t <= FLIGHTS_REDUCED:
        fcase = np.random.binomial(S, FLIGHTS)
        findex = rand.sample(range(POP), fcase)
        findex = df.loc[(df.index.isin(findex))&(df['status']=='S')].index
        if len(findex):
            df.loc[findex, ['status','t']] = [['I',t] for x in findex]
    
    df.loc[(TIME_DEATH <= t - df['t']) & (df['death']=='D'), 'status']='D'
    df.loc[(TIME_RECOVER <= t - df['t']) & (df['death']=='R'), 'status']='R'
    

plt.legend(['Infected','Susceptible','Recovered','Asymptomatic','Hospitalized','Death'])
plt.xlabel('Time')
plt.ylabel('Number of people')

In [ ]:
fig=plt.figure(figsize=(12, 7) )
ax = sns.lineplot(x="t", y="Infected", hue="age"
                  , estimator=None, lw=1,
                  data=result)